# Focus and Collimation using LatissCWFSAlign SAL script
The script is built for the scriptQueue but can be run via this notebook

This script is used to both collimated and focus the telescope. <br>
The script itself is not used to slew to a target, however, it is included in this procedure as an optional cell

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

In [ ]:
from lsst.ts.observatory.control.utils import RotType

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

## Instantiate the Script

First derive and index for the script that is specific to your user

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index=os.getuid()*10+np.random.randint(0,9)
logger.info(f'The generated index is {index}')

Instantiate the script then start all remotes

In [ ]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

Set the script to have a DEBUG log level

In [ ]:
script.log.level = logging.DEBUG

### Optional: Slew to a target
This is not part of the script but seems to come in handy, so it is here for convenience

Print the available instrument filter and gratings. <br>
This is useful both for slewing and for the configuration steps below

In [ ]:
inst_setup = await script.latiss.get_available_instrument_setup()
logger.info(f'filters are: {inst_setup[0]},\ngratings are: {inst_setup[1]}')

In [ ]:
await script.atcs.find_target(az=225, el=70, mag_limit=8)

Uncomment the following line to slew

In [ ]:
target='HD 22132'
await script.atcs.slew_object(target, rot_type=RotType.PhysicalSky)

In [ ]:
await script.atcs.reset_offsets()

Uncomment the following line to take a snapshot to verify the target is positioned as expected 

In [ ]:
await script.latiss.take_engtest(5, filter='FELH0600', grating='empty_1')

Uncomment the following line to offset the telescope (if required). <br>
Offsets are in detector X/Y coordinates and in arcseconds

In [ ]:
# await script.atcs.offset_xy(x=20, y=20)

In [ ]:
await script.atcs.rem.ataos.cmd_offset.set_start(x=1, y=1, z=0.05)

## Configure the script

In [ ]:
configuration = yaml.safe_dump({"filter": 'FELH0600', 
                                "grating": 'empty_1',
                                "exposure_time": 20,})

Set script state to UNCONFIGURED. <br>
The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included. <br>
Therefore, it is included here despite being a non-operation in the first instance.

In [ ]:
script.set_state(ScriptState.UNCONFIGURED)

#### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images

In [ ]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script
This sets the same group ID for all exposures taken in the script

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)
await script.arun()

## Focusing and collimation are now complete

If required, then uncomment and use the following cell to stop the telescope from tracking, but then you will lose your acquisition

In [ ]:
# await script.atcs.stop_tracking()

## Optional Display of donuts and centroids
This is useful for viewing the donuts and centroids. It is primarily used if the script fails.

In [ ]:
%matplotlib inline

In [ ]:
fig1 = plt.figure(1, figsize=(12,8))
ax11 = fig1.add_subplot(121)
ax11.set_title(f"intra visitID - {script.intra_visit_id}")
ax11.imshow(script.I1[0].image0)
ax11.contour(script.algo.pMask) 
ax12 = fig1.add_subplot(122)
ax12.set_title(f"extra visitID - {script.extra_visit_id}")
ax12.imshow(script.I2[0].image0)
ax12.contour(script.algo.pMask) 